In [ ]:
!pip install networkx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Import Package & Data

In [68]:
import networkx as nx
import plotly.graph_objects as go
import numpy as np
import pandas as pd
nc = pd.read_csv('./nc.csv')
nid = pd.read_csv('./nid.csv')
nod = pd.read_csv('./nod.csv')
nc = nc.fillna(0)
gn = pd.read_csv('./garch_network.csv')
del gn['Unnamed: 0']
gn = gn.set_index(['from','to'])
del gn['index']
gn['sum'] = 0

for i in range(0,len(gn.index)):
  gn['sum'][i] = gn.iloc[i,:].sum()
gn = gn.dropna()
nc_mean = pd.DataFrame(nc.mean(),columns=['mean'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Sector Clustering

In [74]:
sector = ['Smart Contract','Smart Contract','Platform','DeFi','Bitcoin Related','Centralized Exchange','Bitcoin Related','Bitcoin Related','Platform','Centralized Exchange',
 'Stable Coin','Meme','Smart Contract','Ethereum Related','Ethereum Related','Platform','Centralized Exchange','DeFi','Centralized Exchange','DeFi',
 'Bitcoin Related','Platform','Centralized Exchange','Platform','DeFi','Platform','Platform','Stable Coin','DeFi',
 'Stable Coin','Stable Coin','Smart Contract','Bitcoin Related','Smart Contract','Privacy','Platform','Platform','Privacy']

sector2 = ['Smart Contract','Smart Contract','Platform','DeFi','Centralized Exchange','Bitcoin Related','Bitcoin Related','Platform','Centralized Exchange',
 'Stable Coin','Meme','Smart Contract','Ethereum Related','Ethereum Related','Platform','DeFi','Centralized Exchange','DeFi',
 'Bitcoin Related','Platform','Centralized Exchange','Platform','DeFi','Platform','Platform','Stable Coin','DeFi',
 'Stable Coin','Stable Coin','Smart Contract','Bitcoin Related','Smart Contract','Privacy','Platform','Platform','Privacy']

In [75]:
coinsector = pd.DataFrame([nc_mean.index,sector]).T
coinsector.columns = ['coin','sector']
nc_mean['sector'] = sector
nc_mean = pd.DataFrame(nc_mean.groupby('sector')['mean'].mean(),columns = ['mean'])
nc_mean

,mean
sector,
Bitcoin Related,14.890569
Centralized Exchange,14.062989
DeFi,12.320463
Ethereum Related,14.667705
Meme,14.420819
Platform,14.783116
Privacy,13.895463
Smart Contract,20.294840
Stable Coin,17.939502


In [69]:
gn = gn.reset_index()
gn = gn.join(coinsector.set_index('coin')['sector'],on='from')
gn = gn.rename(columns = {'sector':'sec_from'})
gn = gn.join(coinsector.set_index('coin')['sector'],on='to')
gn = gn.rename(columns = {'sector':'sec_to'})
gn = gn.set_index(['sec_from','sec_to'])
gn = pd.DataFrame(gn.groupby(['sec_from','sec_to'])['sum'].mean())
gn.head()

sum
sec_from        sec_to                        
Bitcoin Related Bitcoin Related       178.1875
                Centralized Exchange  394.3600
                DeFi                  424.8000
                Ethereum Related      268.1000
                Meme                  489.8000

### Create Nodes and Edges

In [76]:
G = nx.Graph()
for i in range(0,len(nc_mean.index)):
  G.add_node(nc_mean.index[i],size = nc_mean['mean'][i])

G.nodes

NodeView(('Bitcoin Related', 'Centralized Exchange', 'DeFi', 'Ethereum Related', 'Meme', 'Platform', 'Privacy', 'Smart Contract', 'Stable Coin'))

In [77]:
for i in range(0,len(gn.index)):
  G.add_edge(gn.index[i][0],gn.index[i][1],weight = gn['sum'][i])

G.edges

EdgeView([('Bitcoin Related', 'Bitcoin Related'), ('Bitcoin Related', 'Centralized Exchange'), ('Bitcoin Related', 'DeFi'), ('Bitcoin Related', 'Ethereum Related'), ('Bitcoin Related', 'Meme'), ('Bitcoin Related', 'Platform'), ('Bitcoin Related', 'Privacy'), ('Bitcoin Related', 'Smart Contract'), ('Bitcoin Related', 'Stable Coin'), ('Centralized Exchange', 'Centralized Exchange'), ('Centralized Exchange', 'DeFi'), ('Centralized Exchange', 'Ethereum Related'), ('Centralized Exchange', 'Meme'), ('Centralized Exchange', 'Platform'), ('Centralized Exchange', 'Privacy'), ('Centralized Exchange', 'Smart Contract'), ('Centralized Exchange', 'Stable Coin'), ('DeFi', 'DeFi'), ('DeFi', 'Ethereum Related'), ('DeFi', 'Meme'), ('DeFi', 'Platform'), ('DeFi', 'Privacy'), ('DeFi', 'Smart Contract'), ('DeFi', 'Stable Coin'), ('Ethereum Related', 'Ethereum Related'), ('Ethereum Related', 'Meme'), ('Ethereum Related', 'Platform'), ('Ethereum Related', 'Privacy'), ('Ethereum Related', 'Smart Contract'), (

### X,Y Coordinate

In [78]:
pos = nx.spring_layout(G)

In [79]:
for i in range(0,len(nc_mean.index)):
  G.add_node(nc_mean.index[i],size = nc_mean['mean'][i],pos = pos[nc_mean.index[i]])

G.nodes

NodeView(('Bitcoin Related', 'Centralized Exchange', 'DeFi', 'Ethereum Related', 'Meme', 'Platform', 'Privacy', 'Smart Contract', 'Stable Coin'))

### Visualizing

In [80]:
from plotly.colors import n_colors #컬러 스케일 만드는 패키지
colors = n_colors('rgb(0, 36, 58)', 'rgb(252, 106, 151)', len(nc_mean.index), colortype = 'rgb')
color = nc_mean.sort_values('mean')
color['colors'] = colors
color = color.sort_index()
colors = color['colors'].tolist()

size = []
for node in G.nodes():
  size_ = G.nodes[node]['size']
  size.append(size_)

weight = []
for edge in G.edges():
  weight_ = G.edges[edge]['weight']
  weight.append(weight_)

In [81]:
edge_trace=[]
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    trace = go.Scatter(
      x=[x0,x1,None], y=[y0, y1, None],
      line=dict(width=G.edges[edge]['weight']/100, color='cornflowerblue'),
      hoverinfo='none',
      mode='lines')
    edge_trace.append(trace)

node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=nc_mean.index, textfont=dict(size=8),
    marker=dict(
        color=colors,
        line_width=0,
        size=size*1000))

In [82]:
fig = go.Figure()
for trace in edge_trace:
  fig.add_trace(trace)
fig.add_trace(node_trace)
fig.update_layout(
        title = dict(text = 'Causal Network Graph of <br>Crypto Sector Risk-spillovers Normalized Network Connectivity',font = dict(size=15,color='white'),x=0.05,y=0.95),
        autosize = False,
        showlegend = False,
        font = dict(size = 8, color = 'white'),
        plot_bgcolor = '#0d0c1d',paper_bgcolor='#0d0c1d',
        width=800,height=550,margin=dict(b=20,l=10,r=10,t=45),
        xaxis = dict( showline = False, showgrid = False, showticklabels = False, 
                    linecolor = 'white', linewidth = 0,zeroline=False),
        yaxis = dict(showline = False, showgrid = False, showticklabels = False, 
                    linecolor = 'white', linewidth = 0,zeroline=False),
        annotations=[
        dict(xref='paper',yref='paper',x=1,y=0,align='right',
            text='Source: yahoofinance',showarrow=False,font=dict(size=8,color='white')),
        dict(xref='paper',yref='paper',x=1,y=-0.015,align='right',
            text='Data period: 2021.02.03 - 2022.08.18',showarrow=False,font=dict(size=8,color='white')),
        dict(xref='paper',yref='paper',x=1,y=1.05,align='right',
            text='Information',showarrow=False,font=dict(size=8,color='grey')),
        dict(xref='paper',yref='paper',x=1,y=1.035,align='right',
            text='Linkedin: ~/chacehkk',showarrow=False,font=dict(size=8,color='grey')),
        dict(xref='paper',yref='paper',x=1,y=1.02,align='right',
            text='Medium: ~/@hyksun2015',showarrow=False,font=dict(size=8,color='grey'))],

        )
fig.show()


In [ ]:
#nod = nod.fillna(0)
nid_mean = pd.DataFrame(nid.mean(),columns=['mean'])
nid_mean['sector'] = sector2
nid_mean = pd.DataFrame(nid_mean.groupby('sector')['mean'].mean(),columns = ['mean'])

G = nx.Graph()
for i in range(0,len(nid_mean.index)):
  G.add_node(nid_mean.index[i],size = nid_mean['mean'][i])

for i in range(0,len(gn.index)):
  G.add_edge(gn.index[i][0],gn.index[i][1],weight = gn['sum'][i])

pos = nx.spring_layout(G)
for i in range(0,len(nid_mean.index)):
  G.add_node(nid_mean.index[i],size = nid_mean['mean'][i],pos = pos[nid_mean.index[i]])

colors = n_colors('rgb(0, 36, 58)', 'rgb(252, 106, 151)', len(nid_mean.index), colortype = 'rgb')
color = nid_mean.sort_values('mean')
color['colors'] = colors
color = color.sort_index()
colors = color['colors'].tolist()

edge_trace=[]
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    trace = go.Scatter(
      x=[x0,x1,None], y=[y0, y1, None],
      line=dict(width=G.edges[edge]['weight']/100, color='cornflowerblue'),
      hoverinfo='none',
      mode='lines')
    edge_trace.append(trace)

node_x = []
node_y = []
for node in G.nodes():
    x = pos[node][0]
    y = pos[node][1]
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=nid_mean.index, textfont=dict(size=8),
    marker=dict(
        color=colors,
        line_width=0,
        size=nid_mean['mean'].tolist()))
fig = go.Figure()
for trace in edge_trace:
  fig.add_trace(trace)
fig.add_trace(node_trace)

fig.update_layout(
        title = dict(text = 'Causal Network Graph of <br>Crypto Sector Risk-spillovers Network In Degree',font = dict(size=15,color='white'),x=0.05,y=0.95),
   #     legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=0.93),
        autosize = False,
        showlegend = False,
        font = dict(size = 8, color = 'white'),
        plot_bgcolor = '#0d0c1d',paper_bgcolor='#0d0c1d',
        width=800,height=550,margin=dict(b=20,l=10,r=10,t=45),
        xaxis = dict( showline = False, showgrid = False, showticklabels = False, 
                    linecolor = 'white', linewidth = 0,zeroline=False),
        yaxis = dict(showline = False, showgrid = False, showticklabels = False, 
                    linecolor = 'white', linewidth = 0,zeroline=False),
        annotations=[
        dict(xref='paper',yref='paper',x=1,y=0,align='right',
            text='Source: yahoofinance',showarrow=False,font=dict(size=8,color='white')),
        dict(xref='paper',yref='paper',x=1,y=-0.015,align='right',
            text='Data period: 2021.02.03 - 2022.08.18',showarrow=False,font=dict(size=8,color='white')),
        dict(xref='paper',yref='paper',x=1,y=1.05,align='right',
            text='Information',showarrow=False,font=dict(size=8,color='grey')),
        dict(xref='paper',yref='paper',x=1,y=1.035,align='right',
            text='Linkedin: ~/chacehkk',showarrow=False,font=dict(size=8,color='grey')),
        dict(xref='paper',yref='paper',x=1,y=1.02,align='right',
            text='Medium: ~/@hyksun2015',showarrow=False,font=dict(size=8,color='grey'))],

        )
fig.show()